In [1]:
import numpy as np
import pandas as pd
import glob
import sys,os
from pathlib import Path
from PIL import Image
from tqdm import tqdm

### 0.API

In [2]:
sys.path.append('../../')
from utils.plots.plots import draw_bbox

In [3]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

In [4]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}

In [5]:
#  dataframe multi-thread
from pandarallel import pandarallel
# Initialization
pandarallel.initialize()
# df.parallel_apply(func, progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
label2model = get_list2dict( model_label_map.values(), model_label_map.keys())

### 1.Dataset Information.

In [7]:
root_label6 = Path('/home/chenyi/workspace/dataset/labels/labels6')
!tree -d -L 2 $root_label6

/home/chenyi/workspace/dataset/labels/labels6
├── dada_det
├── DeepFashion2
│   └── csv
├── deploy_system
│   └── labels_202207
├── fashionpedia
│   ├── csv
│   └── labels
├── gallery2m
│   ├── gallery2m_7k_test10k
│   ├── gallery2m_7k_train
│   └── gallery7k_100sample
├── haowei34k
│   ├── csv
│   └── labels
├── labels_exp
│   ├── csv
│   ├── dadetGallv4_del
│   └── delBbox
└── wardrobe500k

20 directories


In [9]:
root_data = Path('/home/chenyi/workspace/dataset/data/')
!tree -d -L 2 $root_data

/home/chenyi/workspace/dataset/data
├── classification
│   ├── clean_data
│   ├── clean_shoe
│   ├── error
│   └── gallery2m_v6
├── dada_det
│   ├── hard_sample_0610
│   ├── shoe_bag11k_may
│   ├── shoeBag5k0515
│   └── wholebody0601_all
├── data_fashionpedia
│   ├── annotations
│   ├── test
│   └── train
├── DeepFashion2
│   ├── json_for_test
│   ├── json_for_validation
│   ├── test
│   ├── train
│   └── validation
├── deploy_system
│   ├── images
│   └── images_rename
├── gallery2m
│   └── gallery2m_7k_20220715
├── haowei34k
│   ├── haowei23
│   └── images
└── wardrobe500k
    ├── wardrobe20k_img_hard20220705
    └── wardrobe500k_hard

31 directories


In [8]:
root_yolo = Path('/home/chenyi/workspace/yolov5/datasets/')
!tree -d -L 1 $root_yolo

/home/chenyi/workspace/yolov5/datasets
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dadetv5
├── dadetv5_query
├── dadetv6_1
├── dadetv6_2
├── dadetv6_3
├── dataExps
├── deepedia
├── gallery_train
├── haowei_bag_shoe
├── hwpedia
├── hwpedia70k
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

22 directories


In [7]:
!ls $root_data/bboxes

dadet100k     @eaDir  fashionpedia  hwpedia  tuneDataset
deepfashion2  exps    haowei34k     query


In [29]:
# !mkdir $root_data/data/bboxes

#### csv

In [9]:
root_nas = Path('/nas/chenyi/datasets_nas/gallery2m_clred/gallery2m500_45_20220805/')
root_xml = root_nas /'xml/gallery2m500_45_xml_20220805/'
# !ls $root_xml

##### bboxes_df

In [23]:
bboxes_df = pd.read_csv('/nas/chenyi/datasets_nas/csv_dataset/gallery/gallery2m_v6_bboxes_df.csv')
# bboxes_df = pd.read_csv(root_nas /f'{folder_name}_bboxes_crops_df.csv')
# bboxes_df.to_csv(root_nas /f'{folder_name}_bboxes_crops_df.csv', index=False)

### 解析label.txt

In [9]:
def read_txt(p):
    with open(str(p), 'r') as f:
        label_lines = f.readlines()
    return label_lines

In [10]:
def get_bboxes_info(p):
    import numpy as np
    lines_list = []
    for ind,line in enumerate(read_txt(p)):
        line_list = line.split('\n')[0].split(' ')
        line_list.append(ind)
        line_list.append(p)
        lines_list.append(line_list)
    return lines_list

In [11]:
global bboxes_all
def get_bboxes_multithread(p):
    bboxes_info = get_bboxes_info(p)
    bboxes_all += bboxes_info

In [ ]:
folder_name = 'dadetv6_3'

In [ ]:
root_dataset = Path(f'/home/chenyi/workspace/yolov5/datasets/{folder_name}/labels/*/*.txt')

In [ ]:
labels_p = glob.glob(str(root_dataset))
len(labels_p)

122800

In [ ]:
from tqdm import tqdm
bboxes_all = []
for p in tqdm(labels_p):
    bboxes_info = get_bboxes_info(p)
    bboxes_all += bboxes_info
    # break

100%|███████████████████████████████████████████████████████████████| 122800/122800 [00:28<00:00, 4362.89it/s]


In [ ]:
# %%time
# # global bboxes_all
# # bboxes_all = []
# run_task_multithread(labels_df['crops_p'][:10], get_bboxes_multithread, 20)

#### 处理bbox信息

In [17]:
bboxes_df = pd.DataFrame(bboxes_all)

In [23]:
bboxes_df.head(1)

,label_model,bx1,bx2,bx3,bx4,bbox_ind,label_sp
0,0,0.44003,0.904297,0.13943,0.060547,0,/home/chenyi/workspace/yolov5/datasets/dadetv6...


In [19]:
name_stand = ['label_model', 'bx1', 'bx2', 'bx3', 'bx4', 'bbox_ind', 'label_sp']
bboxes_df.rename(columns={bboxes_df.columns[0]:name_stand[0],
                          bboxes_df.columns[1]:name_stand[1],
                          bboxes_df.columns[2]:name_stand[2],
                          bboxes_df.columns[3]:name_stand[3],
                          bboxes_df.columns[4]:name_stand[4],
                          bboxes_df.columns[5]:name_stand[5],
                          bboxes_df.columns[6]:name_stand[6],
                         },
                          inplace=True)

In [22]:
corners = ['bx1', 'bx2', 'bx3', 'bx4']
for c in corners:
    bboxes_df[c] = bboxes_df[c].astype(float)
bboxes_df['label_model'] = bboxes_df['label_model'].astype(int)
bboxes_df['bbox_ind'] = bboxes_df['bbox_ind'].astype(int)

In [24]:
len(bboxes_df)

332874

In [25]:
bboxes_df['stem'] = bboxes_df['label_sp'].apply(lambda x: Path(x).stem)

In [26]:
bboxes_df['mode'] = bboxes_df['label_sp'].apply(lambda x: Path(x).parts[-2])

In [27]:
bboxes_df.value_counts('mode')

mode
train    289422
val       43452
dtype: int64

##### 去除不要的bbox

In [ ]:
slink_pedia_df = pd.read_csv()

In [173]:
# bboxes_df['label_tp'] = bboxes_df['label_sp'].apply(lambda x: '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/labels/{0}/{1}'.format(Path(x).parts[-2], Path(x).name))

In [175]:
# bboxes_df['img_tp'] = bboxes_df['img_sp'].apply(lambda x: '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/images/{0}/{1}'.format(Path(x).parts[-2], Path(x).name))

In [153]:
# bboxes_df['mode'] = bboxes_df['label_tp'].apply(lambda x: Path(x).parts[-2])

In [159]:
bboxes_df['img_sp'] = bboxes_df['label_tp'].apply(lambda x: f'/home/chenyi/workspace/dataset/data//{x}')
bboxes_df['img_sp'] = bboxes_df['img_sp'].apply(lambda x: x.replace('.txt', '.jpg'))

In [154]:
# bboxes_df['label_sp'] = bboxes_df['label_tp'].apply(lambda x: os.readlink(x))

#### 获取宽高

In [31]:
folder_name = 'gallery2mv6'

In [32]:
root_dataset = Path(f'/home/chenyi/workspace/yolov5/datasets/gallery_train/{folder_name}')
imgs_list = glob.glob(str(root_dataset /'images/*/*.jpg'))
len(imgs_list)

86170

In [33]:
imgs_df = pd.DataFrame()
imgs_df['img_sp'] = imgs_list
imgs_df['stem'] = imgs_df['img_sp'].apply(lambda x: Path(x).stem)

In [34]:
stem2img_sp = get_list2dict(imgs_df['stem'].values, imgs_df['img_sp'].values)

In [35]:
bboxes_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,bbox_ind,label_sp,stem
0,0,0.44003,0.904297,0.13943,0.060547,0,/home/chenyi/workspace/yolov5/datasets/gallery...,551df69f15ca760f19dd6f40bfda2aa7


In [36]:
bboxes_df['img_sp'] = bboxes_df['stem'].map(stem2img_sp)

In [37]:
w_list = []
h_list = []
for img_sp in tqdm(bboxes_df['img_sp'].values):
    w, h = Image.open(img_sp).size
    w_list.append(w)
    h_list.append(h)
    # break

 ... (more hidden) ...


In [38]:
bboxes_df['width'] = w_list
bboxes_df['height'] = h_list

In [43]:
# bboxes_df.to_csv('/nas/chenyi/datasets_nas/csv_dataset/gallery/gallery2m_v6_bboxes_df.csv', index=False)


### 裁切crop图片

In [39]:
sys.path.append('../../')
from utils.bbox.conversation import bbox_yolo2voc

In [40]:
def bbox_yolo2voc_paral(line):
    bbox = line[:4]
    w, h = line[4], line[5]
    x_c = float(bx[0])*w
    y_c = float(bx[1])*h
    wbbox = float(bx[2])*w
    hbbox = float(bx[3])*h
    x1 = int(x_c - wbbox/2)
    y1 = int(y_c - hbbox/2)
    x2 = int(x_c + wbbox/2)
    y2 = int(y_c + hbbox/2)
    bbox = [x1, y1, x2, y2]
    bboxes_list.append(bbox4)
    # return bbox

In [41]:
bboxes_list = []
for ind,row in tqdm(bboxes_df.iterrows()):
    bbox7 = row.loc[['label_pred', 'bx1', 'bx2', 'bx3', 'bx4', 'width', 'height']].values.tolist()
    bbox5 = bbox7[:6]
    bbox4 = bbox_yolo2voc(bbox5[1:], bbox7[5], bbox7[6])
    # bbox5_str = '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(*bbox5)
    bboxes_list.append(bbox4)
    # break

 ... (more hidden) ...


In [ ]:
# bboxes_list = []
# _ = bboxes_df.

In [42]:
bboxes_np = np.array(bboxes_list)

In [86]:
len(bboxes_np[:,0])

295916

In [44]:
bboxes_df['bx1'] = bboxes_np[:, 0]
bboxes_df['bx2'] = bboxes_np[:, 1]
bboxes_df['bx3'] = bboxes_np[:, 2]
bboxes_df['bx4'] = bboxes_np[:, 3]

In [45]:
bboxes_df['bbox_ind'] = bboxes_df['bbox_ind'].astype(int)

In [46]:
bboxes_df['img_train_sp'] = bboxes_df['img_sp'].apply(lambda x: os.readlink(x))

In [47]:
bboxes_df['link'] =  bboxes_df['img_train_sp'].apply(lambda x: Path(x).is_symlink())

In [48]:
bboxes_link_df = bboxes_df[bboxes_df['link']]

In [49]:
bboxes_df.loc[bboxes_link_df.index, 'img_train_sp'] = bboxes_link_df['img_train_sp'].apply(lambda x: os.readlink(x))

In [50]:
bboxes_df['link'] =  bboxes_df['img_train_sp'].apply(lambda x: Path(x).is_symlink())

In [51]:
bboxes_df.sort_values('stem', inplace=True)

In [55]:
bboxes_df.reset_index(drop=True, inplace=True)

In [86]:
bboxes_df.to_csv('/nas/chenyi/datasets_nas/csv_dataset/gallery/gallery2m_v6_crop_df.csv', index=False)

In [100]:
haowei_df.value_counts('label_pred').sort_index()

label_pred
0    61668
1    20515
2    28089
3    18702
4     7625
5     7367
dtype: int64

In [99]:
haowei_df = bboxes_df[bboxes_df['source']=='haowei34k']

In [68]:
bboxes_df['source'] = bboxes_df['img_train_sp'].apply(lambda x: Path(x).parts[-3])

In [84]:
bboxes_df['mode'] = bboxes_df['label_sp'].apply(lambda x: Path(x).parts[-2])

In [85]:
bboxes_df.value_counts('mode')

mode
train    261668
val       34248
dtype: int64

In [73]:
bboxes_df.loc[bboxes_df[bboxes_df['source']=='images'].index, 'source'] = 'haowei34k'

In [95]:
bboxes_df.columns

Index(['label_pred', 'bx1', 'bx2', 'bx3', 'bx4', 'bbox_ind', 'label_sp',
       'stem', 'img_sp', 'width', 'height', 'img_train_sp', 'link', 'source',
       'mode'],
      dtype='object')

In [94]:
import os
write_df = bboxes_df

In [92]:
bbox_ind

6

In [97]:
import shutil
import cv2

# root_source = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/gall7kv6/images/')
root_target = Path('/nas/chenyi/datasets_nas/classification/gallery2m_v6_bakeup')
except_list = []
folder_num = 256
folder_counter = [0]*6
for ind, row in tqdm(write_df.iterrows()):
    sp = Path(row['img_train_sp'])
    bbox = row[['bx1', 'bx2', 'bx3', 'bx4']].values
    bbox_ind = row['bbox_ind']
    catg = int(row['label_pred'])
    mode = row['mode']
    source = row['source']
    name = '{0}_{1}.jpg'.format(row['stem'], bbox_ind)
    tp = root_target /source /mode/ str(catg) /str(folder_counter[catg]//folder_num) /name

    if tp.is_file():
        continue
    if not tp.parent.is_dir():
        tp.parent.mkdir(parents=True, exist_ok=True)
    try:
        img_cv = cv2.imread(str(sp))
        img_crop =  img_cv[bbox[1]:bbox[3], bbox[0]:bbox[2], :]
        cv2.imwrite(str(tp), img_crop)
        folder_counter[catg] += 1
    except:
        except_list.append(sp)
    # break

 ... (more hidden) ...


KeyboardInterrupt: 

In [90]:
!ls '/nas/chenyi/datasets_nas/classification/gallery2m_v6_bakeup/data_fashionpedia/train/3/0/'

train


### target label

In [26]:
!tree -d -L 2 /home/chenyi/workspace/dataset/labels/labels6/

/home/chenyi/workspace/dataset/labels/labels6/
├── fashionpedia
│   └── labels
├── haowei34k
│   └── labels
├── labels_exp
│   ├── dadetGallv4_del
│   └── delBbox
├── train
├── val
└── wardrobe500k
    └── labels10k_20220629

11 directories


In [29]:
!ls /nas/chenyi/datasets_nas/labels/

dada_det  labels6


In [31]:
root_labels = Path('/nas/chenyi/datasets_nas/labels/wardrobe500k/wardrobe20k_labels6')

In [30]:
# !mkdir -p /nas/chenyi/datasets_nas/labels/wardrobe500k/wardrobe20k_labels6

In [412]:
# df = pd.read_csv('./data/exps/dadetGallv4_del.csv')
df = bboxes_df

In [413]:
df.head(1)

,label_pred,bx1,bx2,bx3,bx4,bbox_ind,label_sp,stem,label_tp,mode,img_sp,source,img_tp
2,3,0.828336,0.753906,0.103448,0.101562,2,/home/chenyi/workspace/dataset/labels/labels6/...,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...


In [414]:
len(df)

306304

In [517]:
df.value_counts('label_pred').sort_index()

label_pred
0     59841
1     15162
2    107728
3     49284
4     22571
5     51718
dtype: int64

#### source label path

In [32]:
!ls $root_labels

In [534]:
root = '/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels'

In [535]:
df['label_sp'] = df.apply(lambda row: Path('{0}/{1}/{2}.txt'.format(root_labels, row['mode'], row['stem'])), axis=1)

#### 生成txt

In [10]:
sys.path.append('../../')

In [11]:
from utils.bbox.conversation import bbox_coco2voc,bbox_voc2yolo,bbox_yolo2voc

In [521]:
# df['label_model'] = df['label_model'].map(label2model)

In [12]:
df = bboxes_df

In [14]:
df.head(1)

,xml_path,width,height,labelimg,bx1,bx2,bx3,bx4,stem,img_sp,label86
0,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,1080,1554,裙子,6,478,1080,1129,2017022320122485968,/nas/chenyi/datasets_nas/wardrobe20k_labelimg/...,4


In [17]:
df['label_model'] = df['label86'].values

In [20]:
df.loc[df[df['label_model']==6].index, 'label_model'] = 2
df.loc[df[df['label_model']==7].index, 'label_model'] = 3

In [21]:
df.value_counts('label_model').sort_index()

label_model
0     574
1     751
2    8313
3    4904
4    4670
5    1759
dtype: int64

In [24]:
# yolo
bboxes_list = []
for ind,row in tqdm(df.iterrows()):
    bbox7 = row.loc[['label_model', 'bx1', 'bx2', 'bx3', 'bx4', 'width', 'height']].values.tolist()
    bbox5 = bbox7[:6]
    bbox5[1:] = bbox_voc2yolo(bbox5[1:], bbox7[5], bbox7[6])
    bbox5_str = '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(*bbox5)
    bboxes_list.append(bbox5_str)
    # break

20971it [00:04, 5067.98it/s]


In [438]:
# voc2yolo
# bboxes_list = []
# for ind,row in tqdm(df.iterrows()):
#     bbox7 = row.loc[['label_pred', 'bx1', 'bx2', 'bx3', 'bx4', 'width', 'height']].values.tolist()
#     bbox5 = bbox7[:6]
#     # bbox5[1:] = bbox_voc2yolo(bbox5[1:], bbox7[5], bbox7[6])
#     bbox5_str = '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(*bbox5)
#     bboxes_list.append(bbox5_str)
#     # break

In [23]:
bboxes_list[-1]

'4 0.502778 0.517053 0.994444 0.418919\n'

In [25]:
def write_txt(label_path, text):
    # label_path, text = input_list[0], input_list[1]
    label_path = Path(label_path)
    p_parent = label_path.parent
    if not p_parent.is_dir():
        p_parent.mkdir(parents=True)
    if not p_parent.is_file():
        with open(label_path, 'a+') as f:
            f.write(text)

In [538]:
Path(df['label_sp'].values[0]).parents[1]

PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels')

In [ ]:
# import shutil
# if Path(df['label_sp'].values[0]).parents[1].is_dir():
#     shutil.rmtree(Path(df['label_sp'].values[0]).parents[1])

In [540]:
for label_path, text in tqdm(zip(df['label_sp'].values, bboxes_list)):
    write_txt(label_path, text)
    # break

180695it [00:04, 37924.59it/s]


In [541]:
label_path,text

(PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels/val/2018041313545637741.txt'),
 '0 0.306250 0.525781 0.393750 0.620313\n')

In [542]:
!cat $label_path

0 0.712500 0.465625 0.325000 0.631250
0 0.306250 0.525781 0.393750 0.620313


#### 生成软连接的path

In [36]:
!tree -d -L 1 /home/chenyi/workspace/dataset/data/

/home/chenyi/workspace/dataset/data/
├── dada_det
├── data_fashionpedia
├── deepedia
├── DeepFashion2
├── haowei34k
└── tar

6 directories


In [37]:
!tree -d -L 1 /home/chenyi/workspace/yolov5/datasets/

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

17 directories


In [36]:
# !mkdir -p /home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox

In [543]:
df = bboxes_df

In [544]:
len(df)

180695

In [545]:
df.head(1).values

array([[3, 0.828336, 0.753906, 0.103448, 0.101562, 2,
        PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels/train/551df69f15ca760f19dd6f40bfda2aa7.txt'),
        '551df69f15ca760f19dd6f40bfda2aa7',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/labels/train/551df69f15ca760f19dd6f40bfda2aa7.txt',
        'train',
        '/home/chenyi/workspace/dataset/data/data_fashionpedia/train/551df69f15ca760f19dd6f40bfda2aa7.jpg',
        'data_fashionpedia',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/images/train/551df69f15ca760f19dd6f40bfda2aa7.jpg']],
      dtype=object)

In [546]:
# shbg_df['img_new_sp'] = shbg_df['img_sp'].apply(lambda x: Path(x).parents[1]/Path(x).name)

In [547]:
slink_df.value_counts('source')

source
data_fashionpedia    45529
haowei34k            33399
dada_det             14939
DeepFashion2          4814
shoeBag5k0515         4430
dtype: int64

In [548]:
drop_df = df.drop_duplicates('label_sp')

In [549]:
slink_df = pd.DataFrame()
slink_df['source'] = drop_df['source']
slink_df['fname'] = drop_df['stem']
slink_df['label_sp'] = drop_df['label_sp']
slink_df['img_sp'] = drop_df['img_sp']
slink_df['label_tp'] = drop_df['label_tp']
slink_df['img_tp'] = drop_df['img_tp']

In [41]:
# root = Path('/home/chenyi/workspace/yolov5/datasets/hw_pedia_true/')
# img_root = Path('/home/chenyi/workspace/dataset//')
# yolo_data_root = Path('/home/chenyi/workspace/yolov5/datasets/dataExps/gall10k_delBox/')

In [323]:
# slink_df['label_tp'] = slink_df['label_sp'].apply(lambda x: yolo_data_root /'labels' /x.parts[-2]/x.parts[-1])

In [43]:
slink_df['img_tp'] = slink_df['label_sp'].apply(lambda x: yolo_data_root /'images' /x.parts[-2]/x.parts[-1])
slink_df['img_tp'] = slink_df['img_tp'].apply(lambda x: x.with_suffix('.jpg'))

In [44]:
# slink_df['img_sp'] = slink_df['img_tp'].apply(lambda x: data_root /'images' /x.parts[-2]/x.parts[-1])

In [290]:
slink_df['img_file'] = slink_df['img_sp'].apply(lambda x: Path(x).is_file())
slink_df['label_file'] = slink_df['label_sp'].apply(lambda x: Path(x).is_file())

In [550]:
slink_df.to_csv('/nas/chenyi/workspace/myPyModule/dataset/data/bboxes/dadet100k/dadetv5_query_slink.csv', index=False)

#### 直接读取slink.csv

In [ ]:
# slink_df = pd.read_csv('./data/hw34k_level1_slink.csv')

##### 生成label_tp, img_sp, img_tp

In [551]:
slink_df.tail(2).values

array([['shoeBag5k0515', '2017050718162854629',
        PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels/val/2017050718162854629.txt'),
        '/home/chenyi/workspace/dataset/data/dada_det/shoeBag5k0515/images/2017050718162854629.jpg',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/labels/val/2017050718162854629.txt',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/images/val/2017050718162854629.jpg'],
       ['shoeBag5k0515', '2018041313545637741',
        PosixPath('/home/chenyi/workspace/dataset/labels/query6/dadetv5_query/labels/val/2018041313545637741.txt'),
        '/home/chenyi/workspace/dataset/data/dada_det/shoeBag5k0515/images/2018041313545637741.jpg',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/labels/val/2018041313545637741.txt',
        '/home/chenyi/workspace/yolov5/datasets/dadet100kv4_1_june/images/val/2018041313545637741.jpg']],
      dtype=object)

In [552]:
origin_folder = 'dadet100kv4_1_june'
folder_name = 'dadetv5_query'

In [553]:
slink_df['img_tp'] = slink_df['img_tp'].apply(lambda x: str(x).replace(f'/datasets/{origin_folder}/images/', f'/datasets/{folder_name}/images/'))
slink_df['label_tp'] = slink_df['label_tp'].apply(lambda x: str(x).replace(f'/datasets/{origin_folder}/labels/', f'/datasets/{folder_name}/labels/'))

In [554]:
slink_df.to_csv('/nas/chenyi/workspace/myPyModule/dataset/data/bboxes/dadet100k/dadetv5_query_slink.csv', index=False)

### 生成sym link

In [555]:
root = f'/home/chenyi/workspace/yolov5/datasets/'
!tree  -d -L 1 $root

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dadetv5
├── dadetv5_query
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia70k
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

20 directories


In [556]:
folder_name = 'dadetv5_query'

In [382]:
# 产生路径
import shutil
# folder_name = 'gall10k_delBox'
folder_name = folder_name
folder_root = Path(root) /folder_name
# folder_root = root
folder1 = ['images', 'labels']
folder2 = ['train', 'val']
# if folder_root.is_dir() and True:
#     shutil.rmtree(folder_root)
for f1 in folder1:
    for f2 in folder2:
        tg = folder_root / f1 / f2
        tg.mkdir(parents=True)  

In [459]:
!tree  -d -L 1 $folder_root

/home/chenyi/workspace/yolov5/datasets/dadetv5_query
├── images
└── labels

2 directories


In [558]:
for p in glob.glob(str(Path(slink_df['img_tp'].values[0]).parents[1]/'*/*.jpg')):
    if Path(p).is_symlink():
        Path(p).unlink()

In [559]:
for p in glob.glob(str(Path(slink_df['label_tp'].values[0]).parents[1]/'*/*.txt')):
    if Path(p).is_symlink():
        Path(p).unlink()

In [560]:
slink_df.head(1)

,source,fname,label_sp,img_sp,label_tp,img_tp
2,data_fashionpedia,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/dataset/labels/query6/d...,/home/chenyi/workspace/dataset/data/data_fashi...,/home/chenyi/workspace/yolov5/datasets/dadetv5...,/home/chenyi/workspace/yolov5/datasets/dadetv5...


In [561]:
%%time
for p in slink_df['img_tp'].values:
    if Path(p).is_symlink():
        Path(p).unlink()
_ = slink_df.apply(lambda row: Path.symlink_to(Path(row['img_tp']), Path(row['img_sp'])), axis=1)

CPU times: user 2.25 s, sys: 1.18 s, total: 3.42 s
Wall time: 3.6 s


In [562]:
%%time
for p in slink_df['label_tp'].values:
    if Path(p).is_symlink():
        Path(p).unlink()
_ = slink_df.apply(lambda row: Path.symlink_to(Path(row['label_tp']), Path(row['label_sp'])), axis=1)

CPU times: user 2.36 s, sys: 1.17 s, total: 3.53 s
Wall time: 4.05 s


In [563]:
len(glob.glob(str(Path(slink_df['img_tp'].values[0]).parents[1]/'*/*.jpg'))), len(glob.glob(str(Path(slink_df['label_tp'].values[0]).parents[1]/'*/*.txt')))

(103109, 103109)

In [564]:
p = str(slink_df['label_tp'].values[1001])
!cat $p

5 0.572266 0.614370 0.367188 0.768328


In [565]:
p = str(slink_df['label_sp'].values[2501])
!cat $p

5 0.573206 0.473633 0.209370 0.460938


#### 核查软连接

In [566]:
import glob,cv2, os
import pandas as pd

In [567]:
!tree  -d -L 1 /home/chenyi/workspace/yolov5/datasets/

/home/chenyi/workspace/yolov5/datasets/
├── coco128
├── dadet100kv2
├── dadet100kv3_june
├── dadet100kv4_1_june
├── dadet100kv4_june
├── dadet_gallv4_june
├── dadetv5
├── dadetv5_query
├── dataExps
├── deepedia
├── deepedia_catg6
├── deepedia_catg8
├── haowei_bag_shoe
├── hwpedia
├── hwpedia70k
├── hwpedia80k
├── tuneGallery10k
├── tuneGallery5k
├── tuneQuery10k
└── validation_all

20 directories


In [568]:
# folder_name = 'dadet100kv2'
root = f'/home/chenyi/workspace/yolov5/datasets/{folder_name}'
!tree  -d -L 2 $root

/home/chenyi/workspace/yolov5/datasets/dadetv5_query
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories


In [569]:
def check_syslink(root, mode='images'):
    subfix = '.txt' if mode=='labels' else '.jpg'
    flag_list = [os.path.isfile(os.readlink(p)) for p in glob.glob(f'{root}/{mode}/*/*{subfix}')]
    txt_df = pd.DataFrame()
    txt_df['flag'] = flag_list
    return txt_df.value_counts('flag')

In [570]:
%%time
check_syslink(root, mode='images')

CPU times: user 201 ms, sys: 236 ms, total: 437 ms
Wall time: 495 ms


flag
True    103109
dtype: int64

In [571]:
%%time
check_syslink(root, mode='labels')

CPU times: user 190 ms, sys: 193 ms, total: 383 ms
Wall time: 381 ms


flag
True    103109
dtype: int64